<h1>
<center>
Dataquest Guided Project 18:
Credit Modelling
</center>
</h1>

# Introduction

This is part of the Dataquest program.

- part of paths **Data Scientist in Python**
    - Step 6: **Machine Learning**
        - Course 7 :  **Machine Learning Walkthrough**
            
As this is a guided project, we are following and deepening the steps suggested by Dataquest. In this project, we will walk through the full data science life cycle, from data cleaning and feature selection to machine learning.

# Use case : Credit Modelling : Lending Club

## How Lending Club works

We will focus on credit modeling, a well-known data science problem that focuses on modeling a borrower's credit risk.
We'll be working with financial lending data from [Lending Club]( https://www.lendingclub.com/). Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return. 

Each borrower fills out a comprehensive application, providing their past financial history, the reason for the loan, and more. Lending Club evaluates each borrower's credit score using previous historical data (and their data science process!) and assigns an interest rate to the borrower. The interest rate is the percent in addition to the requested loan amount the borrower has to pay back. 

A higher interest rate means that the borrower is riskier and more unlikely to pay back the loan while a lower interest rate states that the borrower has a good credit history is more likely to pay back the loan. The interest rates range from 5.32% all the way to 30.99%, and each borrower is given a grade according to the interest rate they were assigned. If the borrower accepts the interest rate, then the loan is listed on the Lending Club marketplace.

Investors are primarily interested in receiving a return on their investments. Approved loans are listed on the Lending Club website, where qualified investors can browse recently approved loans, the borrower's credit score, the purpose for the loan, and other information from the application. Once they're ready to back a loan, they select the amount of money they want to fund. Once a loan's requested amount is fully funded, the borrower receives the money they asked for minus the origination fee that Lending Club charges.

The borrower then makes monthly payments back to Lending Club either over 36 months or over 60 months. Lending Club redistributes these payments to the investors. This means that investors don't have to wait until the full amount is paid off to start to see the money back. If a loan is fully paid off on time, the investors make a return which corresponds to the interest rate the borrower had to pay in addition to the requested amount. Many loans aren't entirely paid off on time, however, and some borrowers default on the loan.

Here's a diagram from [Bible Money Matters](http://www.biblemoneymatters.com/introduction-to-peer-to-peer-lending-signing-up-to-use-lending-club/) that sums up the process:

![Howitworks](http://cdn.biblemoneymatters.com/wp-content/uploads/2009/08/how-social-lending-works.jpg)

While Lending Club has to be very savvy and rigorous with their credit modeling, investors on Lending Club need to be equally as savvy about determining which loans are more likely to be paid off. If investors believe the borrower can pay back the loan, even if he or she has a weak financial history, then investors can make more money through the larger additional amount the borrower has to pay.

Most investors use a portfolio strategy to invest small amounts in many loans, with healthy mixes of low, medium, and interest loans. In this course, we'll focus on the mindset of a conservative investor who only wants to invest in the loans that have a good chance of being paid off on time. To do that, we'll need first to understand the features in the dataset and then experiment with building machine learning models that reliably predict if a loan will be paid off or not.

## Introduction to the data

Lending Club releases data for all of the approved and declined loan applications periodically on [their website](https://www.lendingclub.com/info/download-data.action). We can select a few different years ranges to download the datasets (in CSV format) for both approved and declined loans.

We can also find a data dictionary (in XLS format) which contains information on the different column names towards the bottom of the page hosted on [Google Drive](https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit).

The **LoanStats** sheet describes the approved loans datasets and the **RejectStats** describes the rejected loans datasets. Since rejected applications don't appear on the Lending Club marketplace and aren't available for investment, we'll be focusing on data on approved loans only.

The approved loans datasets contain information on current loans, completed loans, and defaulted loans. Let's now define the problem statement for this machine learning project:

**Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?**

Before we can start doing machine learning, we need to define what features we want to use and which column represents the target column we want to predict. Let's start by reading in the dataset and exploring it.

In this mission, we'll focus on approved loans data from 2007 to 2011, since a good number of the loans have already finished. In the datasets for later years, many of the loans are current and still being paid off.

# Data Cleaning

In [1]:
import pandas as pd
from __future__ import division 

## Reducing the number of columns

In [2]:
loans_2007 = pd.read_csv("loans_2007.csv")
loans_2007.drop_duplicates()
print(loans_2007.iloc[0])
print(loans_2007.shape[1])

id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                          Dec-2011
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

/Users/work/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


We want to pay attention to any features that:

- leak information from the future (after the loan has already been funded)
- don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
- formatted poorly and need to be cleaned up
- require more data or a lot of processing to turn into a useful feature
- contain redundant information

We need to especially pay attention to data leakage, since it can cause our model to overfit. This is because the model would be using data about the target column that wouldn't be available when we're using the model on future loans. 

After analyzing each column in the [data dictionary](https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit), we can conclude that the following features need to be removed:

- **id**: randomly generated field by Lending Club for unique identification purposes only
- **member_id** : also a randomly generated field by Lending Club for unique identification purposes only
- **funded_amnt**: leaks data from the future (after the loan is already started to be funded)
- **funded_amnt_inv**: also leaks data from the future (after the loan is already started to be funded)
- **grade**: contains redundant information as the interest rate column (int_rate)
- **sub_grade**: also contains redundant information as the interest rate column (int_rate)
- **emp_title**: requires other data and a lot of processing to potentially be useful
- **issue_d**: leaks data from the future (after the loan is already completed funded)
- **zip_code**: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
- **out_prncp**: leaks data from the future, (after the loan already started to be paid off)
- **out_prncp_inv**: also leaks data from the future, (after the loan already started to be paid off)
- **total_pymnt**: also leaks data from the future, (after the loan already started to be paid off)
- **total_pymnt_inv**: also leaks data from the future, (after the loan already started to be paid off)
- **total_rec_prncp**: also leaks data from the future, (after the loan already started to be paid off)
- **total_rec_int**: leaks data from the future, (after the loan already started to be paid off),
- **total_rec_late_fee**: also leaks data from the future, (after the loan already started to be paid off),
- **recoveries**: also leaks data from the future, (after the loan already started to be paid off),
- **collection_recovery_fee**: also leaks data from the future, (after the loan already started to be paid off),
- **last_pymnt_d**: also leaks data from the future, (after the loan already started to be paid off),
- **last_pymnt_amnt**: also leaks data from the future, (after the loan already started to be paid off).

In [3]:
cols_to_drop = ["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d",
                "zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp",
                "total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", 
                "last_pymnt_amnt"]
loans_2007 = loans_2007.drop(cols_to_drop, axis=1)
print(loans_2007.iloc[0])
print(loans_2007.shape[1])

loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                          83.7%
total_acc                               9
initial_list_status                     f
last_credit_pull_d               J

Just by becoming familiar with the columns in the dataset, we were able to reduce the number of columns from 52 to 32 columns. We now need to decide on a target column that we want to use for modeling.

## Preparing the target column

We should use the loan_status column, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. Currently, this column contains text values and we need to convert it to a numerical one for training a model. Let's explore the different values in this column and come up with a strategy for converting the values in this column.

In [4]:
print(loans_2007['loan_status'].value_counts())

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64


There are 8 different possible values for the loan_status column: 

|**Loan Status**|**Count**|**Meaning**|
|---------------|---------|-----------|
|Fully Paid|33136|Loan has been fully paid off|
|Charged Off|5634|Loan for which there is no longer a reasonable expectation of further payments|
|Does not meet the credit policy. Status:Fully Paid|1988|While the loan was paid off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace|
|Does not meet the credit policy. Status:Charged Off|761|While the loan was charged off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace|
|In Grace Period|20|The loan is past due but still in the grace period of 15 days|
|Late (16-30 days)|8|Loan hasn't been paid in 16 to 30 days (late on the current payment)|
|Late (31-120 days)|24|Loan hasn't been paid in 31 to 120 days (late on the current payment)|
|Current|961|Loan is up to date on current payments|
|Default|3|Loan is defaulted on and no payment has been made for more than 121 days|

From the investor's perspective, we're interested in trying to predict which loans will be paid off on time and which ones won't be. Only the Fully Paid and Charged Off values describe the final outcome of the loan. The other values describe loans that are still on going and where the jury is still out on if the borrower will pay back the loan on time or not. While the Default status resembles the Charged Off status, in Lending Club's eyes, loans that are charged off have essentially no chance of being repaid while default ones have a small chance. 

Since we're interested in being able to predict which of these 2 values a loan will fall under, we can treat the problem as a binary classification one. Let's remove all the loans that don't contain either Fully Paid and Charged Off as the loan's status and then transform the Fully Paid values to 1 for the positive case and the Charged Off values to 0 for the negative case.

Lastly, one thing we need to keep in mind is the class imbalance between the positive and negative cases. While there are 33,136 loans that have been fully paid off, there are only 5,634 that were charged off. This class imbalance is a common problem in binary classification and during training, the model ends up having a strong bias towards predicting the class with more observations in the training set and will rarely predict the class with less observations. The stronger the imbalance, the more biased the model becomes. There are a few different ways to tackle this class imbalance, which we'll explore later

In [5]:
loans_2007 = loans_2007[(loans_2007['loan_status'] == "Fully Paid") | (loans_2007['loan_status'] == "Charged Off")]

status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}

loans_2007 = loans_2007.replace(status_replace)

### Removing Single value columns

Let's look for any columns that contain only one unique value and remove them. These columns won't be useful for the model since they don't add any information to each loan application.

In [6]:
columns = loans_2007.columns

cols_to_drop = []
for col in columns:
    unique_values = loans_2007[col].dropna().unique()
    if len(unique_values) == 1:
        cols_to_drop.append(col)
        
loans_2007 = loans_2007.drop(cols_to_drop, axis=1)
print(cols_to_drop)

['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


It looks we we were able to remove 9 more columns since they only contained 1 unique value.

# Preparing the features

We'll now prepare the data for machine learning by focusing on handling missing values, converting categorical columns to numeric columns, and removing any other extraneous columns; we encounter throughout this process.

This is because the mathematics underlying most machine learning models assumes that the data is numerical and contains no missing values. To reinforce this requirement, scikit-learn will return an error if you try to train a model using data that include missing values or non-numeric values when working with models like linear regression and logistic regression.

## Handling Missing Values

Let's start by computing the number of missing values and come up with a strategy for handling them. Then, we'll focus on the categorical columns.

In [7]:
null_counts = loans_2007.isnull().sum()
print(null_counts)

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64


While most of the columns have 0 missing values, 2 columns have 50 or less rows with missing values, and 1 column, pub_rec_bankruptcies, contains 697 rows with missing values. Let's remove columns entirely where more than 1% of the rows for that column contain a null value. In addition, we'll remove the remaining rows containing null values.

This means that we'll keep the following columns and just remove rows containing missing values for them:

- title
- revol_util
- last_credit_pull_d

and drop the pub_rec_bankruptcies column entirely since more than 1% of the rows have a missing value for this column.




In [8]:
loans_2007 = loans_2007.drop("pub_rec_bankruptcies", axis=1)
loans_2007 = loans_2007.dropna(axis=0)
print(loans_2007.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


## Text columns

While the numerical columns can be used natively with scikit-learn, the object columns that contain text need to be converted to numerical data types. Let's return a new Dataframe containing just the object columns so we can explore them in more depth. 

In [9]:
object_columns_df = loans_2007.select_dtypes(include=["object"])
print(object_columns_df.iloc[0])

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Jun-2016
Name: 0, dtype: object


Some of the columns seem like they represent categorical values, but we should confirm by checking the number of unique values in those columns:

- **home_ownership** : home ownership status, can only be 1 of 4 categorical values according to the data dictionary,
- **verification_status**: indicates if income was verified by Lending Club,
- **emp_length**: number of years the borrower was employed upon time of application,
- **term**: number of payments on the loan, either 36 or 60,
- **addr_state**: borrower's state of residence,
- **purpose**: a category provided by the borrower for the loan request,
- **title**: loan title provided the borrower,

There are also some columns that represent numeric values, that need to be converted:

- **int_rate**: interest rate of the loan in %,
- **revol_util**: revolving line utilization rate or the amount of credit the borrower is using relative to all available credit, read more here.

Based on the first row's values for purpose and title, it seems like these columns could reflect the same information. Let's explore the unique value counts separately to confirm if this is true.

Lastly, some of the columns contain date values that would require a good amount of feature engineering for them to be potentially useful:

- **earliest_cr_line**: The month the borrower's earliest reported credit line was opened,
- **last_credit_pull_d**: The most recent month Lending Club pulled credit for this loan.

Since these date features require some feature engineering for modeling purposes, let's remove these date columns from the Dataframe.

In [10]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for c in cols:
    print('*****' + c + '*******')
    print(loans_2007[c].value_counts())

*****home_ownership*******
RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
*****verification_status*******
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
*****emp_length*******
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64
*****term*******
 36 months    28234
 60 months     9441
Name: term, dtype: int64
*****addr_state*******
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
LA     420
AL     420
KY     311
OK     285
KS     249
UT     249
AR     229
DC 

The home_ownership, verification_status, emp_length, term, and addr_state columns all contain multiple discrete values. We should clean the emp_length column and treat it as a numerical one since the values have ordering (2 years of employment is less than 8 years).

First, let's look at the unique value counts for the purpose and title columns to understand which column we want to keep.

In [11]:
print(loans_2007["purpose"].value_counts())
print(loans_2007["title"].value_counts())

debt_consolidation    17751
credit_card            4911
other                  3711
home_improvement       2808
major_purchase         2083
small_business         1719
car                    1459
wedding                 916
medical                 655
moving                  552
house                   356
vacation                348
educational             312
renewable_energy         94
Name: purpose, dtype: int64
Debt Consolidation                          2068
Debt Consolidation Loan                     1599
Personal Loan                                624
Consolidation                                488
debt consolidation                           466
Credit Card Consolidation                    345
Home Improvement                             336
Debt consolidation                           314
Small Business Loan                          298
Credit Card Loan                             294
Personal                                     290
Consolidation Loan                       

### Categorical columns

The home_ownership, verification_status, emp_length, and term columns each contain a few discrete categorical values. We should encode these columns as dummy variables and keep them.

It seems like the purpose and title columns do contain overlapping information but we'll keep the purpose column since it contains a few discrete values. In addition, the title column has data quality issues since many of the values are repeated with slight modifications (e.g. Debt Consolidation and Debt Consolidation Loan and debt consolidation).

We can use the following mapping to clean the emp_length column:

- "10+ years": 10
- "9 years": 9
- "8 years": 8
- "7 years": 7
- "6 years": 6
- "5 years": 5
- "4 years": 4
- "3 years": 3
- "2 years": 2
- "1 year": 1
- "< 1 year": 0
- "n/a": 0

We erred on the side of being conservative with the 10+ years, < 1 year and n/a mappings. We assume that people who may have been working more than 10 years have only really worked for 10 years. We also assume that people who've worked less than a year or if the information is not available that they've worked for 0. This is a general heuristic but it's not perfect.

Lastly, the addr_state column contains many discrete values and we'd need to add 49 dummy variable columns to use it for classification. This would make our Dataframe much larger and could slow down how quickly the code runs. Let's remove this column from consideration.

In [12]:
mapping = {
    "emp_length":
    {"10+ years": 10,
     "9 years": 9,
     "8 years": 8,
     "7 years": 7,
     "6 years": 6,
     "5 years": 5,
     "4 years": 4,
     "3 years": 3,
     "2 years": 2,
     "1 year": 1,
     "< 1 year": 0,
     "n/a": 0}
}


cat_to_drop = ["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"]

loans_2007 = loans_2007.drop(cat_to_drop, axis=1)
loans_2007["int_rate"] = loans_2007["int_rate"].str.rstrip("%").astype("float")
loans_2007["revol_util"] = loans_2007["revol_util"].str.rstrip("%").astype("float")
loans_2007 = loans_2007.replace(mapping)

## Dummy variables

Let's now encode the home_ownership, verification_status, purpose, and term columns as dummy variables so we can use them in our model.

In [13]:
cat_columns = ["home_ownership", "verification_status", "purpose", "term"]

dummies = pd.get_dummies(loans_2007[cat_columns])
loans_2007 = pd.concat([loans_2007, dummies], axis=1)
loans_2007 = loans_2007.drop(cat_columns, axis=1)

# Making Predictions 

## Picking an error metric

An error metric will help us figure out when our model is performing well, and when it's performing poorly. To tie error metrics all the way back to the original question we wanted to answer, let's say we're using a machine learning model to predict whether or not we should fund a loan on the Lending Club platform. Our objective in this is to make money -- we want to fund enough loans that are paid off on time to offset our losses from loans that aren't paid off. An error metric will help us determine if our algorithm will make us money or lose us money.

In this case, we're primarily concerned with false positives and false negatives. Both of these are different types of misclassifications. With a false positive, we predict that a loan will be paid off on time, but it isn't. This costs us money since we fund loans that lose us money. With a false negative, we predict that a loan won't be paid off on time, but it actually would be paid off on time. This loses us potential money since we didn't fund a loan that actually would have been paid off.

In the loan_status and prediction columns, a 0 means that the loan wouldn't be paid off on time, and a 1 means that it would.

Here's a diagram to simplify the concepts:


|actual loan_status|prediction|error type|
|------------------|----------|----------|
| 0                | 1        | False Positive|
| 1                | 1        | True Positive|
| 0                | 0        | True Negative|
| 1                | 0        | False Negative|


Since we're viewing this problem from the standpoint of a conservative investor, we need to treat false positives differently than false negatives. A conservative investor would want to minimize risk, and avoid false positives as much as possible. They'd be more okay with missing out on opportunities (false negatives) than they would be with funding a risky loan (false positives).

We can calculate false positive rate and true positive rate, using the numbers of true positives, true negatives, false negatives, and false positives.

False positive rate is the number of false positives divided by the number of false positives plus the number of true negatives. This divides all the cases where we thought a loan would be paid off but it wasn't by all the loans that weren't paid off. 

![rates](rates.png)

In this case, we should optimize for:

- high recall (true positive rate)
- low fall-out (false positive rate)



## Logistic Regression

A good first algorithm to apply to binary classification problems is logistic regression, for the following reasons:

- it's quick to train and we can iterate more quickly,
- it's less prone to overfitting than more complex models like decision trees,
- it's easy to interpret.

### Creating the model

In [14]:
# Let's prepare the features/target columns :

cols = loans_2007.columns
train_cols = cols.drop("loan_status")
features = loans_2007[train_cols]

target = loans_2007["loan_status"]

In [15]:
from sklearn.linear_model import LogisticRegression

In [16]:
lr = LogisticRegression()
lr.fit(features, target)
predictions = lr.predict(features)
predictions

array([1, 1, 1, ..., 1, 1, 1])

### K-fold validation

While we generated predictions in the last screen, those predictions were overfit. They were overfit because we generated predictions using the same data that we trained our model on. When we use this to evaluate error, we get an unrealistically high depiction of how accurate the algorithm is, because it already "knows" the correct answers.

In order to get a realistic depiction of the accuracy of the model, let's perform k-fold cross validation. 

In [17]:
from sklearn.model_selection import cross_val_predict
lr = LogisticRegression()

predictions = cross_val_predict(lr, features, target)
predictions = pd.Series(predictions)

Let's compute the false positive rate and true positive rate : 

In [18]:
# False positives.
fp_filter = (predictions == 1) & (loans_2007["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans_2007["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans_2007["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans_2007["loan_status"] == 0)
tn = len(predictions[tn_filter])

# True positive rate 
tpr = tp / (tp + fn)

# False negative rate 
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.998426848453
0.998617966436


### Penalizing the classifier

Unfortunately, even through we're not using accuracy as an error metric, the classifier is, and it isn't accounting for the imbalance in the classes. There are a few ways to get a classifier to correct for imbalanced classes. The two main ways are:

Use oversampling and undersampling to ensure that the classifier gets input that has a balanced number of each class.
Tell the classifier to penalize misclassifications of the less prevalent class more than the other class.

We'll look into oversampling and undersampling first. They involve taking a sample that contains equal numbers of rows where loan_status is 0, and where loan_status is 1. This way, the classifier is forced to make actual predictions, since predicting all 1s or all 0s will only result in 50% accuracy at most.

The downside of this technique is that since it has to preserve an equal ratio, you have to either:

- Throw out many rows of data. If we wanted equal numbers of rows where loan_status is 0 and where loan_status is 1, one way we could do that is to delete rows where loan_status is 1.
- Copy rows multiple times. One way to equalize the 0s and 1s is to copy rows where loan_status is 0.
- Generate fake data. One way to equalize the 0s and 1s is to generate new rows where loan_status is 0.

Unfortunately, none of these techniques are especially easy. The second method we mentioned earlier, telling the classifier to penalize certain rows more, is actually much easier to implement using scikit-learn. By setting the class_weight parameter to balanced, the penalty is set to be inversely proportional to the class frequencies. This would mean that for the classifier, correctly classifying a row where loan_status is 0 is 6 times more important than correctly classifying a row where loan_status is 1.

In [19]:
lr = LogisticRegression(class_weight="balanced")
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans_2007["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans_2007["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans_2007["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans_2007["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.636307026744
0.621915103653


We significantly improved false positive rate in the last screen by balancing the classes, which reduced true positive rate. Our true positive rate is now around 67%, and our false positive rate is around 40%. From a conservative investor's standpoint, it's reassuring that the false positive rate is lower because it means that we'll be able to do a better job at avoiding bad loans than if we funded everything. However, we'd only ever decide to fund 67% of the total loans (true positive rate), so we'd immediately reject a good amount of loans.

We can try to lower the false positive rate further by assigning a harsher penalty for misclassifying the negative class. While setting class_weight to balanced will automatically set a penalty based on the number of 1s and 0s in the column, we can also set a manual penalty. 

In [20]:
manual_penalty = {
    0:10,
    1:1
}

lr = LogisticRegression(class_weight=manual_penalty)
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans_2007["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans_2007["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans_2007["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans_2007["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.225714472994
0.222902270484


## Random Forests

Let's now try a more complex algorithm, random forest. Random forests are able to work with nonlinear data, and learn complex conditionals. 
Logistic regressions are only able to work with linear data. Training a random forest algorithm may enable us to get more accuracy due to columns that correlate nonlinearly with loan_status.

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
rf = RandomForestClassifier(class_weight="balanced", random_state=1)

predictions = cross_val_predict(rf, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans_2007["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans_2007["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans_2007["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans_2007["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)

0.963096486628
0.963474827246


# Conclusions

Unfortunately, using a random forest classifier didn't improve our false positive rate. The model is likely weighting too heavily on the 1 class, and still mostly predicting 1s. We could fix this by applying a harsher penalty for misclassifications of 0s.

Ultimately, our best model had a false positive rate of 22%, and a true positive rate of 22%. For a conservative investor, this means that they make money as long as the interest rate is high enough to offset the losses from 22% of borrowers defaulting, and that the pool of 22% of borrowers is just large enough to make enough interest money to offset the losses.

Given this, there's still quite a bit of room to improve:

- We can tweak the penalties further.
- We can try models other than a random forest and logistic regression.
- We can use some of the columns we discarded to generate better features.
- We can ensemble multiple models to get more accurate predictions.
- We can tune the parameters of the algorithm to achieve higher performance.